In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
train_dir = r'/content/drive/MyDrive/Osteoarthritis_Assignment_dataset/Osteoarthritis_Assignment_dataset/train'
validation_dir = r'/content/drive/MyDrive/Osteoarthritis_Assignment_dataset/Osteoarthritis_Assignment_dataset/Valid'
test_dir =r'/content/drive/MyDrive/Osteoarthritis_Assignment_dataset/Osteoarthritis_Assignment_dataset/test'

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 2350 images belonging to 2 classes.
Found 641 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.applications import InceptionV3

conv_base = InceptionV3(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
conv_base.summary()

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 3, 3, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 18432)             0         
                                                                 
 dense (Dense)               (None, 256)               4718848   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 26,521,889
Trainable params: 26,487,457
Non-trainable params: 34,432
_________________________________________________________________


In [ ]:
# conv_base.trainable = False

In [ ]:
from tensorflow.keras import optimizers

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [ ]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("CNN_Project_Model-{epoch:02d}.h5")

In [29]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=32,
      callbacks=[checkpoint_cb])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys


Epoch 1/10
100/100 [==============================] - 449s 4s/step - loss: 0.4184 - acc: 0.7995 - val_loss: 0.4352 - val_acc: 0.7797
Epoch 2/10
100/100 [==============================] - 410s 4s/step - loss: 0.3965 - acc: 0.8136 - val_loss: 0.8745 - val_acc: 0.7250
Epoch 3/10
100/100 [==============================] - 412s 4s/step - loss: 0.3640 - acc: 0.8342 - val_loss: 0.7859 - val_acc: 0.7609
Epoch 4/10
100/100 [==============================] - 416s 4s/step - loss: 0.3161 - acc: 0.8558 - val_loss: 0.2538 - val_acc: 0.9078
Epoch 5/10
100/100 [==============================] - 415s 4s/step - loss: 0.3198 - acc: 0.8568 - val_loss: 0.2140 - val_acc: 0.9219
Epoch 6/10
100/100 [==============================] - 423s 4s/step - loss: 0.2661 - acc: 0.8769 - val_loss: 0.1511 - val_acc: 0.9438
Epoch 7/10
100/100 [==============================] - 415s 4s/step - loss: 0.2977 - acc: 0.8678 - val_loss: 0.1570 - val_acc: 0.9438
Epoch 8/10
100/100 [==============================] - 418s 4s/step - 

Now you can test this model on any knee x-ray image you want and find out whether the image is of a healthy person or a person with Osteoarthritis condition.

For that, you have to import the libraries given below.

In [30]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img

Enter the location of the previously saved model and load it using the keras.models.load_model command.

In [31]:
model = keras.models.load_model('/content/drive/MyDrive/CNN_Project_Model-10.h5')

Enter the location of knee x - ray image that you wants to test and load the image.

Use model.predict to predict the output.

The output of this block of code will be the probability of the patient having osteoarthritis. Closer is the value to 1, higher the chances of having osteoarthritis.

In [42]:
img=image.load_img('/content/drive/MyDrive/Osteoarthritis_Assignment_dataset/Osteoarthritis_Assignment_dataset/train/Osteoarthritis/DoubtfulG1 (27).png',target_size=(150,150))

x=image.img_to_array(img)

import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
#img_data.shape

Probability = model.predict(img_data)

print(Probability[0][0])


0.998955


In [35]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 845 images belonging to 2 classes.


In [36]:
model.evaluate_generator(test_generator, steps=42)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  """Entry point for launching an IPython kernel.


[13.213926315307617, 0.3761904835700989]

In [ ]:
prob = model.predict(test_generator, 1)
print(prob)
if prob<=0.5:
    out = "Diseased"
    probab = 100*(1-prob)
    remedy = "Apply sulfur sprays or copper-based fungicides weekly at first sign of disease to prevent its spread."
    print(remedy)
else: 
    out = "Healthy"
    probab = 100*prob
    remedy = " "
    print(remedy)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

ValueError: ignored